In [1]:
import random
from copy import deepcopy

RETURN_WEIGHT = 7
RISK_WEIGHT = 1
NUM_OF_PURCHASED_STOCKS_WEIGHT = 0.01

NEEDED_RETURN = 10
HIGHEST_RISK = 0.6
LEAST_STOCKS = 30

class Stock:
    _name: str
    _return: float
    _risk: float

    def __init__(self, n, p, r):
        self._name = n
        self._return = p
        self._risk = r

class Chromososme:
    genes: list[float]

    def __init__(self, g):
        self.genes = g

    def set_sum_to_one(self):
        total = sum(self.genes)
        self.genes = [a/total for a in self.genes]

    def get_num_of_purhased_stocks(self):
        num_of_purchased_stocks = 0

        for i in range(len(self.genes)):
            if self.genes[i] != 0:
                num_of_purchased_stocks += 1
        
        return num_of_purchased_stocks
    
    def fitness_function(self, stocks: list[Stock]):
        fitness = 0

        for i in range(len(self.genes)):
            fitness += self.genes[i] * stocks[i]._return * RETURN_WEIGHT
            fitness -= self.genes[i] * stocks[i]._risk * RISK_WEIGHT

        fitness += self.get_num_of_purhased_stocks() * NUM_OF_PURCHASED_STOCKS_WEIGHT

        return fitness

    def mutate(self, probability):
        for i in range(len(self.genes)):
            if random.uniform(0, 1) < probability:
                self.genes[i] = random.random()

        self.set_sum_to_one()

def uniform_cross_over(parents: list[Chromososme], probability):
    num_of_genes = len(parents[0].genes)

    childs = [Chromososme([0]*num_of_genes), 
              Chromososme([0]*num_of_genes)]
 
    for i in range(num_of_genes):
        for_0 = 1 if random.uniform(0, 1) < probability else 0

        childs[0].genes[i] = parents[for_0].genes[i]
        childs[1].genes[i] = parents[1 - for_0].genes[i]

    childs[0].set_sum_to_one()
    childs[1].set_sum_to_one()

    return childs
    
class PortfolioManagement:
    stocks_information: list[Stock]
    population: list[Chromososme]
    population_size: int
    fitness_scores: list
    num_of_elites: int
    generation_number: int
    mating_pool: list[Chromososme]
    cross_over_rate: int
    cross_over_probability: int
    mutation_rate: int
    mutation_probability: int

    def __init__(self,
                 population_size,
                 num_of_elites,
                 cross_over_rate,
                 cross_over_probability,
                 mutation_rate,
                 mutation_probability):
        
        self.stocks_information = []
        self.population = []
        self.population_size = population_size
        self.fitness_scores = []
        self.num_of_elites = num_of_elites
        self.generation_number = 1
        self.mating_pool = []
        self.cross_over_rate = cross_over_rate
        self.cross_over_probability = cross_over_probability
        self.mutation_rate = mutation_rate
        self.mutation_probability = mutation_probability 

    def is_goal(self, chromosome: Chromososme):
        total_risk = total_return = 0

        for i in range(len(self.stocks_information)):
            total_risk += chromosome.genes[i] * self.stocks_information[i]._risk
            total_return += chromosome.genes[i] * self.stocks_information[i]._return

        if total_return >= NEEDED_RETURN and total_risk <= HIGHEST_RISK and chromosome.get_num_of_purhased_stocks() >= LEAST_STOCKS:
            print("Answer:")
            print("Return:", total_return)
            print("Risk:", total_risk)
            print("Num of Purchased Stocks:", chromosome.get_num_of_purhased_stocks())
            
            return True
        
    def read_input(self, filename):
        with open(filename, 'r') as file:
            next(file)

            for line in file:
                _number, _name, _risk, _return = line.split(',')
                self.stocks_information.append(Stock(_name, float(_return), float(_risk)))

    def generate_first_generation(self):
        num_of_genes = len(self.stocks_information)
        self.population: list[Chromososme] = [None] * self.population_size

        for i in range(self.population_size):
            genes = []
            for j in range(num_of_genes):
                genes.append(random.random())

            self.population[i] = Chromososme(genes)
            self.population[i].set_sum_to_one()

        self.generation_number = 1

    def select_parents_for_mating_pool(self):
        total = ((len(self.population))*(len(self.population)+1))/2
        
        self.mating_pool = []
        for i in range(self.population_size - self.num_of_elites):   
            random_number = int(random.random()*total)
            s_index = 0

            for j in range(len(self.population)):
                s_index += j+1

                if s_index >= random_number:
                    self.mating_pool.append(self.fitness_scores[j][0])
                    break

        random.shuffle(self.mating_pool)
    
    def rank_by_fitness_scores(self):
        self.fitness_scores = []

        for chromosome in self.population:
            self.fitness_scores.append((chromosome, chromosome.fitness_function(self.stocks_information)))

        self.fitness_scores.sort(key = lambda x: x[1])

    def generate_next_generation(self):
        self.rank_by_fitness_scores()
        
        self._print()

        for i in range(-1, -10, -1):
            if self.is_goal(self.fitness_scores[i][0]):
                return self.fitness_scores[i][0]
            
        self.select_parents_for_mating_pool()

        next_population = []

        for i in range(self.population_size-1, self.population_size-self.num_of_elites-1, -1):
            next_population.append(self.fitness_scores[i][0])
        
        for i in range(0, len(self.mating_pool)-1, 2):
            childs = [self.mating_pool[i], self.mating_pool[i+1]]

            if random.uniform(0, 1) < self.cross_over_rate:
                childs = uniform_cross_over([childs[0], childs[1]], self.cross_over_probability)

            if random.uniform(0, 1) < self.mutation_rate:
                childs[0].mutate(self.mutation_probability)

            if random.uniform(0, 1) < self.mutation_rate:
                childs[1].mutate(self.mutation_probability)

            next_population.append(childs[0])
            next_population.append(childs[1])

        self.population = next_population
        self.generation_number += 1

        return None

    def solve(self):
        self.generate_first_generation()

        while True:
            answer = self.generate_next_generation()

            if answer != None:
                return answer
            
    def _print(self):
        print("Generation Number:", self.generation_number)
        print("Best Fitness Score:", self.fitness_scores[-1][1])
        print("Worst Fitness Score:", self.fitness_scores[0][1])
        fitness_sum = sum([x[1] for x in self.fitness_scores])
        print("Average of Fitness Scores:", fitness_sum / len(self.fitness_scores))
        print("------------------------------------------------------")

manager = PortfolioManagement(population_size= 100,
                              num_of_elites= 10,
                              cross_over_rate= 0.5,
                              cross_over_probability= 0.5,
                              mutation_rate= 0.1,
                              mutation_probability= 0.1)

manager.read_input("Dataset/sample.csv")
manager.solve()

Generation Number: 1
Best Fitness Score: 9.464128926549202
Worst Fitness Score: 8.151423762363336
Average of Fitness Scores: 8.869492186559247
------------------------------------------------------
Generation Number: 2
Best Fitness Score: 9.84219613289006
Worst Fitness Score: 6.5671636922020795
Average of Fitness Scores: 8.747848485093213
------------------------------------------------------
Generation Number: 3
Best Fitness Score: 11.6434974232291
Worst Fitness Score: 7.282936760525849
Average of Fitness Scores: 9.071745116410437
------------------------------------------------------
Generation Number: 4
Best Fitness Score: 15.701924711342398
Worst Fitness Score: 5.888005807590453
Average of Fitness Scores: 9.689219479269703
------------------------------------------------------
Generation Number: 5
Best Fitness Score: 14.552804869055276
Worst Fitness Score: 7.282010386760373
Average of Fitness Scores: 9.477066033122812
------------------------------------------------------
Generatio